# Multi Layer (Deep) Neural Network

## Load MNIST dataset

In [6]:
import pandas as pd
import numpy as np

names = ['label']
feature_names = map(str, range(784))
names.extend(feature_names)

df_mnist_train = pd.read_csv('mnist_train.csv', header=None, nrows=1000, names=names)
df_mnist_test = pd.read_csv('mnist_test.csv', header=None, nrows=100, names=names)

# normalize pixel data (otherwise pure python implemention won't work)
max_pixel =  255
df_mnist_train.iloc[:, 1:] /= max_pixel
df_mnist_test.iloc[:, 1:] /= max_pixel


# select subset of features and labels 

X_train, y_train = df_mnist_train.iloc[:, 1:], df_mnist_train['label']
X_test, y_test = df_mnist_test.iloc[:, 1:], df_mnist_test['label']

print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(1000, 784) (1000,)
(100, 784) (100,)


## Part 1: tensorflow implementation

In [9]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

# define model input
def input_fn(x, y):
    feature_cols = {name: tf.constant(x[name].values) for name in x.columns}
    label = tf.constant(y.values)
    return feature_cols, label


# define features
feat_cols = [tf.contrib.layers.real_valued_column(name) for name in X_train.columns]

# instantiate and fit model
model_2 = tf.contrib.learn.DNNClassifier(hidden_units=[100], feature_columns=feat_cols, n_classes=10) # need to specify n_classes
model_2.fit(input_fn=lambda: input_fn(X_train, y_train), steps=100)

# evaluate model
train_acc = model_2.evaluate(input_fn=lambda: input_fn(X_train, y_train), steps=1)['accuracy']
test_acc = model_2.evaluate(input_fn=lambda: input_fn(X_test, y_test), steps=1)['accuracy']
print 'Train accuracy: {}'.format(train_acc)
print 'Test accuracy: {}'.format(test_acc)

Train accuracy: 0.941999971867
Test accuracy: 0.870000004768
